<a href="https://colab.research.google.com/github/KhoaHLA/DS101---Advanced-Probability-and-Statistics/blob/main/TransAnalysis_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bài tập cá nhân \

**Họ tên**: Hoàng Lê Anh Khoa \
**MSSV**: 22520667

In [ ]:
# Install findspark: Adds Pyspark to sys.path at runtime
!pip install -q findspark

# Install pyspark
!pip install pyspark

In [ ]:
import findspark
findspark.init()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from pyspark.sql import SparkSession
spark  = SparkSession.builder.appName('TransAnalysis').getOrCreate()

# TransAnalysis 1

In [ ]:
transRDD = spark.sparkContext.textFile('/content/drive/MyDrive/Big Data/TransAnalysis/trans.txt').map(lambda x: x.split(','))
transRDD.collect()

In [ ]:
custRDD = spark.sparkContext.textFile('/content/drive/MyDrive/Big Data/TransAnalysis/cust.txt').map(lambda x: x.split(','))
custRDD.collect()

In [ ]:
TransAnalyis1 =transRDD.map(lambda cols: (cols[4].strip(), float(cols[3])))\
                            .reduceByKey(lambda x,y: x+y)\
                            .sortBy(lambda x: x[0])
TransAnalyis1.collect()

# TransAnalysis2:

In [ ]:
TransAnalysis2 = transRDD.map(lambda cols: (cols[4].strip(), ([cols[2]], float(cols[3])))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .sortBy(lambda x: x[0])

TransAnalysis2.collect()

# TransAnalysis 3:

In [ ]:
TransAnalysis3 = transRDD \
    .map(lambda cols: (cols[4].strip(), (set([cols[2]]), float(cols[3])))) \
    .reduceByKey(lambda x, y: (x[0].union(y[0]), x[1] + y[1])).sortBy(lambda x: x[0])
TransAnalysis3.collect()

# TransAnalysis 4

In [ ]:
mapRDD = transRDD.map(lambda cols: (cols[4].strip(), (cols[2], float(cols[3]))))
groupRDD = mapRDD.groupByKey()
TransAnalysis4 = groupRDD.mapValues(lambda x: (len(set([i[0] for i in x])), sum([i[1] for i in x]))).sortBy(lambda x: x[0])
TransAnalysis4.collect()

# TransAnalysis 5

In [ ]:
cust_map = custRDD.map(lambda cols: (cols[0], cols[1]))
trans_map = transRDD.map(lambda cols : (cols[2], (cols[4], float(cols[3]))))

In [ ]:
combine_map = trans_map.join(cust_map).map(lambda cols: (cols[1][0][0], (([cols[1][1]]), cols[1][0][1])))
TransAnalysis5 = combine_map.reduceByKey(
    lambda a, b: (
        list(set(a[0] + b[0])),  # Gộp và loại bỏ trùng lặp các tên người chơi
        a[1] + b[1]              # Cộng tổng amount
    )
).sortBy(lambda x: x[0])
TransAnalysis5.collect()

# TransAnalysis 6

In [ ]:
trans_map = transRDD.map(lambda cols : (cols[2], (cols[4], float(cols[3]))))
cust_map = custRDD.map(lambda cols: (cols[0], (cols[1], str(cols[3]))))

In [ ]:
combine_map = trans_map.join(cust_map).map(lambda cols: (cols[1][0][0], ([cols[1][1][0], cols[1][1][1]], cols[1][0][1])))
combine_map.collect()

In [ ]:
def reduce(x,y):
    seen = {i[0] for i in x[0]}  # lưu các name đã có
    distinct_player = x[0] + [i for i in y[0] if i[0] not in seen]
    total_amount = x[1] + y[1]
    return distinct_player, total_amount

In [ ]:
TransAnalysis6 = combine_map.reduceByKey(reduce).sortBy(lambda x: x[0])
TransAnalysis6.collect()